In [ ]:
!pip install rdkit-pypi
!pip install igraph
!pip install descriptastorus
!pip install networkx
!pip install POT
!pip install cython numpy

In [ ]:
import os
import csv
import codecs
import numpy as np
import networkx as nx
import pickle
import torch

from rdkit import Chem
from typing import List, Tuple, Union
from torch_geometric.utils import from_networkx, tree_decomposition

# 3.1 Construction of igraph objects for WWL graph kernels

In [ ]:
#logic: https://www.blopig.com/blog/2022/02/how-to-turn-a-smiles-string-into-a-molecular-graph-for-pytorch-geometric/
#Ref: https://github.com/divelab/MoleculeX/blob/molx/AdvProp/graph/src/tran_data.py
#atom and bond feature is the same as DMPNN: https://github.com/chemprop/chemprop/blob/02a6f76b21fec9fab000c47d8a185a8c19fecbcb/chemprop/features/featurization.py#L642
#lack of following feature
#a2b: A mapping from an atom index to a list of incoming bond indices.
#b2a: A mapping from a bond index to the index of the atom the bond originates from.
#b2revb: A mapping from a bond index to the index of the reverse bond.
#b2br: A mapping from f_bonds to real bonds in molecule recorded in targets.

import igraph as ig

BOND_FDIM = 14

MAX_ATOMIC_NUM = 100
ATOM_FEATURES = {
    'atomic_num': list(range(MAX_ATOMIC_NUM)),
    'degree': [0, 1, 2, 3, 4, 5],
    'formal_charge': [-1, -2, 1, 2, 0],
    'chiral_tag': [0, 1, 2, 3],
    'num_Hs': [0, 1, 2, 3, 4],
    'hybridization': [
        Chem.rdchem.HybridizationType.SP,
        Chem.rdchem.HybridizationType.SP2,
        Chem.rdchem.HybridizationType.SP3,
        Chem.rdchem.HybridizationType.SP3D,
        Chem.rdchem.HybridizationType.SP3D2
    ],
}

# Distance feature sizes
PATH_DISTANCE_BINS = list(range(10))
THREE_D_DISTANCE_MAX = 20
THREE_D_DISTANCE_STEP = 1
THREE_D_DISTANCE_BINS = list(range(0, THREE_D_DISTANCE_MAX + 1, THREE_D_DISTANCE_STEP))


def onek_encoding_unk(value: int, choices: List[int]) -> List[int]:
  """
  Creates a one-hot encoding.
  :param value: The value for which the encoding should be one.
  :param choices: A list of possible values.
  :return: A one-hot encoding of the value in a list of length len(choices) + 1.
  If value is not in the list of choices, then the final element in the encoding is 1.
  """
  encoding = [0] * (len(choices) + 1)
  index = choices.index(value) if value in choices else -1
  encoding[index] = 1

  return encoding


def atom_features(atom: Chem.rdchem.Atom, functional_groups: List[int] = None) -> List[Union[bool, int, float]]:
  """
  Builds a feature vector for an atom.
  :param atom: An RDKit atom.
  :param functional_groups: A k-hot vector indicating the functional groups the atom belongs to.
  :return: A list containing the atom features.
  """
  features = onek_encoding_unk(atom.GetAtomicNum() - 1, ATOM_FEATURES['atomic_num']) + \
              onek_encoding_unk(atom.GetTotalDegree(), ATOM_FEATURES['degree']) + \
              onek_encoding_unk(atom.GetFormalCharge(), ATOM_FEATURES['formal_charge']) + \
              onek_encoding_unk(int(atom.GetChiralTag()), ATOM_FEATURES['chiral_tag']) + \
              onek_encoding_unk(int(atom.GetTotalNumHs()), ATOM_FEATURES['num_Hs']) + \
              onek_encoding_unk(int(atom.GetHybridization()), ATOM_FEATURES['hybridization']) + \
              [1 if atom.GetIsAromatic() else 0] + \
              [atom.GetMass() * 0.01]  # scaled to about the same range as other features

  if functional_groups is not None:
      features += functional_groups
  return features


def bond_features(bond: Chem.rdchem.Bond) -> List[Union[bool, int, float]]:
  """
  Builds a feature vector for a bond.
  :param bond: A RDKit bond.
  :return: A list containing the bond features.
  """
  if bond is None:
      fbond = [1] + [0] * (BOND_FDIM - 1)
  else:
      bt = bond.GetBondType()
      fbond = [
          0,  # bond is not None
          bt == Chem.rdchem.BondType.SINGLE,
          bt == Chem.rdchem.BondType.DOUBLE,
          bt == Chem.rdchem.BondType.TRIPLE,
          bt == Chem.rdchem.BondType.AROMATIC,
          (bond.GetIsConjugated() if bt is not None else 0),
          (bond.IsInRing() if bt is not None else 0)
      ]
      fbond += onek_encoding_unk(int(bond.GetStereo()), list(range(6)))


  return fbond

def smile_to_graph(smile):
  mol = Chem.MolFromSmiles(smile)
  atoms = [atom_features(atom) for atom in mol.GetAtoms()] # collecting all node features
  atom_n= [atom.GetAtomicNum() for atom in mol.GetAtoms()]
  G = nx.Graph()
  G2 = nx.Graph()

  for i in range(len(atoms)):
    G.add_node(i)        # initialize a graph with adding nodes
    G.nodes[i]['x'] = atoms[i] # assign node feature
    G2.add_node(i)
    G2.nodes[i]['label'] = atoms[i]

  for i in range(len(atoms)):
    for j in range(i, len(atoms)):
      bond = mol.GetBondBetweenAtoms(i, j) # initialize edges to previous graph
      if bond:
        # G is a networkx grpah objects for keeping all D-MPNN model features
        G.add_edge(i, j) # adding edges to previous graph
        G.edges[i, j]['edge_attr'] = bond_features(bond) # assign bond feature
        idx= bond.GetIdx()
        # G2 is an igraph objects for WWL graph kernels, the WWL kernels didn't considered bond feature due to the limitation of WWL test
        # See https://github.com/BorgwardtLab/WWL/blob/master/src/wwl/wwl.py for detail
        G2.add_edge(i, j)
        # .NumBondRings(bond.GetIdx()): returns the number of rings that given input: bond idx, is involved in
        # In this study, number of rings (N3/N7 ring mentioned in my thesis) is a potenial feature to affect HLM stabilibty, but WWL graph kernels not yet implement
        # GetRingInfo(): https://www.rdkit.org/docs/cppapi/classRDKit_1_1RingInfo.html
        # NumBondRings: https://www.rdkit.org/docs/cppapi/classRDKit_1_1RingInfo.html#a72f74c9fac23b98e59b64ab27173ff05
        # minBondRingSize: https://www.rdkit.org/docs/cppapi/classRDKit_1_1RingInfo.html#a7a85cc1ab6ea186b3a65435b6037a644
        G2.edges[i, j]['weight'] = bond.GetOwningMol().GetRingInfo().NumBondRings(bond.GetIdx())

  IG_g = ig.Graph.from_networkx(G2)

  return G, IG_g

# 3.2 Converting SMILES dataset to igraph objects

In [ ]:
#Ref: https://github.com/divelab/MoleculeX/blob/molx/AdvProp/graph/src/tran_data.py

# get input(smiles) and labels from yuor dataset
def data_reader(file_name):

  inputs = []
  labels = []

  with codecs.open(file_name, "r", encoding="utf-8-sig") as f:
    reader = csv.DictReader(f)
    for row in reader:
      smiles = row['smiles']
      inputs.append(smiles)
      labels.append([float(row[y]) if row[y] != '' else np.nan for y in row.keys() if y != 'smiles' and y != 'id'])

    return inputs, np.array(labels)


smiles, labels = data_reader('/content/eli_duplcate_hf.csv') # get input(smiles) and labels from yuor dataset

networks_all=[]
igrapg_list_all=[]

for smile in smiles:
  G, I_G = smile_to_graph(smile)
  networks_all.append(G)
  igrapg_list_all.append(I_G)


In [ ]:
!mkdir gml
#save igraphs in gml files for WWL graph kernel inputs
for i in range(len(igrapg_list_all)):
  igrapg_list_all[i].write(f'/content/gml/{i}.gml', format='gml')

# 3.3.1 Utility functions of WWL graph kernels

In [ ]:
# Ref: https://github.com/BorgwardtLab/WWL/blob/master/experiments/utilities.py

import numpy as np
import os
import igraph as ig

#################
# File loaders
#################
def read_labels(filename):
  '''
  Reads labels from a file. Labels are supposed to be stored in each
  line of the file. No further pre-processing will be performed.
  '''
  labels = []
  with open(filename) as f:
      labels = f.readlines()
      labels = [label.strip() for label in labels]

  return labels

def read_gml(filename):
	node_features = []
	g = ig.read(filename)

	if not 'label' in g.vs.attribute_names():
		g.vs['label'] = list(map(str, [l for l in g.vs.degree()]))

	node_features = g.vs['label']
	adj_mat = np.asarray(g.get_adjacency().data)

	return node_features, adj_mat

def retrieve_graph_filenames(data_directory):
  # Load graphs
  files = os.listdir(data_directory)
  graphs = [g for g in files if g.endswith('gml')]
  graphs.sort()

  return [os.path.join(data_directory, g) for g in graphs]

def load_continuous_graphs(data_directory):
  graph_filenames = retrieve_graph_filenames(data_directory)

  # initialize
  node_features = []
  adj_mat = []
  n_nodes = []

  # Iterate across graphs and load initial node features
  for graph_fname in graph_filenames:
    node_features_cur, adj_mat_cur = read_gml(graph_fname)
    # Load features
    node_features.append(np.asarray(node_features_cur).astype(float).reshape(-1,1))
    adj_mat.append(adj_mat_cur.astype(int))
    n_nodes.append(adj_mat_cur.shape[0])

  # Check if there is a node_features.npy file
  # containing continuous attributes
  # PS: these were obtained by processing the TU Dortmund website
  # If none is present, keep degree or label as features.
  attribtues_filenames = os.path.join(data_directory, 'node_features.npy')
  if os.path.isfile(attribtues_filenames):
    node_features = np.load(attribtues_filenames)

  n_nodes = np.asarray(n_nodes)
  node_features = np.asarray(node_features)

  return node_features, adj_mat, n_nodes

def create_adj_avg(adj_cur):
	'''
	create adjacency
	'''
	deg = np.sum(adj_cur, axis = 1)
	deg = np.asarray(deg).reshape(-1)

	deg[deg!=1] -= 1

	deg = 1/deg
	deg_mat = np.diag(deg)
	adj_cur = adj_cur.dot(deg_mat.T).T

	return adj_cur


def create_labels_seq_cont(node_features, adj_mat, h):
	'''
	create label sequence for continuously attributed graphs
	'''
	n_graphs = len(node_features)
	labels_sequence = []
	for i in range(n_graphs):
    graph_feat = []

		for it in range(h+1):
			if it == 0:
				graph_feat.append(node_features[i])
			else:
				adj_cur = adj_mat[i] + np.identity(adj_mat[i].shape[0])
				adj_cur = create_adj_avg(adj_cur)

				np.fill_diagonal(adj_cur, 0)
				graph_feat_cur = 0.5*(np.dot(adj_cur, graph_feat[it-1]) + graph_feat[it-1])
				graph_feat.append(graph_feat_cur)

		labels_sequence.append(np.concatenate(graph_feat, axis = 1))
		if i % 100 == 0:
			print(f'Processed {i} graphs out of {n_graphs}')

	return labels_sequence

# 3.3.2 Main functions of WWL kernel graphs

In [ ]:
# Ref: https://github.com/BorgwardtLab/WWL/blob/master/experiments/wwl.py

import numpy as np
from sklearn.preprocessing import scale
from sklearn.base import TransformerMixin
import argparse
import igraph as ig
import os
import ot

####################
# Embedding schemes
####################
def compute_wl_embeddings_continuous(data_directory, h):
  '''
  Continuous graph embeddings
  TODO: for package implement a class with same API as for WL
  '''
  node_features, adj_mat, n_nodes = load_continuous_graphs(data_directory)

  node_features_data = scale(np.concatenate(node_features, axis=0), axis = 0)
  splits_idx = np.cumsum(n_nodes).astype(int)
  node_features_split = np.vsplit(node_features_data,splits_idx)
  node_features = node_features_split[:-1]

  # Generate the label sequences for h iterations
  labels_sequence = create_labels_seq_cont(node_features, adj_mat, h)

  return labels_sequence

def compute_wasserstein_distance(label_sequences, h, sinkhorn=False, discrete=False, sinkhorn_lambda=1e-2):
  '''
  Generate the Wasserstein distance matrix for the graphs embedded
  in label_sequences
  '''
  # Get the iteration number from the embedding file
  n = len(label_sequences)
  emb_size = label_sequences[0].shape[1]
  n_feat = int(emb_size/(h+1))

  # Iterate over all possible h to generate the Wasserstein matrices
  hs = range(0, h + 1)
  wasserstein_distances = []

  for h in hs:
    M = np.zeros((n,n))
    # Iterate over pairs of graphs
    for graph_index_1, graph_1 in enumerate(label_sequences):
      # Only keep the embeddings for the first h iterations
      labels_1 = label_sequences[graph_index_1][:,:n_feat*(h+1)]
      for graph_index_2, graph_2 in enumerate(label_sequences[graph_index_1:]):
        labels_2 = label_sequences[graph_index_2 + graph_index_1][:,:n_feat*(h+1)]
        # Get cost matrix
        ground_distance = 'hamming' if discrete else 'euclidean'
        costs = ot.dist(labels_1, labels_2, metric=ground_distance)

        if sinkhorn:
            mat = ot.sinkhorn(np.ones(len(labels_1))/len(labels_1),
                                np.ones(len(labels_2))/len(labels_2), costs, sinkhorn_lambda,
                                numItermax=50)
            M[graph_index_1, graph_index_2 + graph_index_1] = np.sum(np.multiply(mat, costs))
        else:
            M[graph_index_1, graph_index_2 + graph_index_1] = \
                ot.emd2([], [], costs)

    M = (M + M.T)
    wasserstein_distances.append(M)
    print(f'Iteration {h}: done.')

  return wasserstein_distances

In [ ]:
import numpy as np
import pandas as pd
import argparse
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

h=10
label_sequences = compute_wl_embeddings_continuous('content/gml', h)
out_name = f'wl_continue_embeddings_{h}.npy'

output_path='/content/wwl_embedding'
np.save(os.path.join(output_path, out_name), label_sequences)


print('Computing the Wasserstein distances...')
wasserstein_distances = compute_wasserstein_distance(label_sequences, h, sinkhorn= False, discrete= False)

!mkdir Wasserstein_distances
output_path2= '/content/Wasserstein_distances'
filext = 'wasserstein_distance_matrix'

for i, D_w in enumerate(wasserstein_distances):
  filext += f'_it_{i}.npy'
  np.save(os.path.join(output_path2,filext), D_w)

print('Wasserstein distances computation done. Saved to file.')

In [ ]:
!zip -r /content/WWL_d.zip /content/Wasserstein_distances

# 3.4 Evaluating results of WWL graph kernels

In [ ]:
from sklearn.model_selection import ParameterGrid, StratifiedKFold
from sklearn.model_selection._validation import _fit_and_score
from sklearn.base import clone
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

def custom_grid_search_cv2(model, param_grid, precomputed_kernels, y, cv=5):
  '''
  Custom grid search based on the sklearn grid search for an array of precomputed kernels
  '''
  # 1. Stratified K-fold
  cv = KFold(n_splits=cv, shuffle=False)
  results = []

  for train_index, test_index in cv.split(precomputed_kernels[0], y):
    split_results = []
    params = [] # list of dict, its the same for every split
    # run over the kernels first
    for K_idx, K in enumerate(precomputed_kernels):
      # Run over parameters
      for p in list(ParameterGrid(param_grid)):
        sc = _fit_and_score(clone(model), K, y, scorer=make_scorer(mean_squared_error,squared= False, greater_is_better= False),
                train=train_index, test=test_index, verbose=0, parameters=p, fit_params=None)
        split_results.append(sc['test_scores'])
        #print(sorted(sc.keys()))
        params.append({'K_idx': K_idx, 'params': p})
    results.append(split_results)
  # Collect results and average
  results = np.array(results)
  fin_results = results.mean(axis=0)
  # select the best results
  best_idx = np.argmin(fin_results)
  # Return the fitted model and the best_parameters
  ret_model = clone(model).set_params(**params[best_idx]['params'])

  return ret_model.fit(precomputed_kernels[params[best_idx]['K_idx']], y), params[best_idx]

In [ ]:
#Ref: https://github.com/BorgwardtLab/WWL/blob/master/experiments/main.py

from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR

gammas = np.logspace(-4,1,num=6)
# iterate over the iterations too
hs = range(h)
param_grid = [
    {'C': np.logspace(-3,3,num=7)}
        ]

kernel_matrices = []
kernel_params = []

for i, current_h in enumerate(hs):
  # Generate the full list of kernel matrices from which to select
  M = wasserstein_distances[current_h]
  for g in gammas:
    K = np.exp(-g*M)
    kernel_matrices.append(K)
    kernel_params.append((current_h, g))


crossvalidation= True
print(f'Running SVMs, crossvalidation: {crossvalidation}, gridsearch: {gridsearch}.')
# Load labels
#label_file = os.path.join(data_path, 'Labels.txt')
y = np.array(read_labels('/content/HF.txt'), dtype= np.float16)

# Contains accuracy scores for each cross validation step; the
# means of this list will be used later on.
rmse_scores = []
np.random.seed(42)

cv = KFold(n_splits=5, shuffle=True)

# Hyperparam logging
best_C = []
best_h = []
best_gamma = []
finm=[]

for i in range(len(kernel_matrices)):
  meanv=[]
  for train_index, test_index in cv.split(kernel_matrices[i], y):
    K_train = [K[train_index][:, train_index] for K in kernel_matrices]
    K_test  = [K[test_index][:, train_index] for K in kernel_matrices]
    y_train, y_test = y[train_index], y[test_index]

    # Gridsearch
    if gridsearch:
        gs, best_params = custom_grid_search_cv2(SVR(kernel='precomputed'),
                param_grid, K_train, y_train, cv=5)
        # Store best params
        C_ = best_params['params']['C']
        h_, gamma_ = kernel_params[best_params['K_idx']]
        y_pred = gs.predict(K_test[best_params['K_idx']])
        #print(y_pred)
    elif gridsearch== False:
        gs = SVR(C=100, kernel='precomputed').fit(K_train[0], y_train)
        y_pred = gs.predict(K_test[0])
        h_, gamma_, C_ = h, gammas[0], 100
    best_C.append(C_)
    best_h.append(h_)
    best_gamma.append(gamma_)

    rmse_scores.append(mean_squared_error(y_test, y_pred, squared=False))
    if not crossvalidation:
      break
    meanv.append(mean_squared_error(y_test, y_pred, squared=False))

  finm.append(np.mean(meanv))

print(finm)